# CompareBoth

In [ ]:
import pymongo

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Tweet"]

user_collection = db["users"]
user_labeled_svm_collection = db["users_labeled"]
user_predicted_svm_collection = db["users_predicted"]
user_predicted_kmeans_collection = db["users_kmeans_labeled"]

user_compare_both_collection = db["users_compare_both"]

In [ ]:
user_compare_both_collection.drop()

## Fusion des résultats du Kmeans avec SVM dans users_compare_both

### Faire une copie des attributs de users dans users_compare_both

In [ ]:
users = user_collection.find({})
user_compare_both_collection.insert_many(users)

### Merge les données labélisées du SVM dans users_compare_both

In [ ]:
pipeline = [
    {
        '$project': {
            '_id': 1,
            'user_id': 1,
            'label_svm': '$label'
        }
    },
    {
        '$merge': {
            'into': 'users_compare_both',
            'on': '_id',
            'whenMatched': 'merge',
            'whenNotMatched': 'discard'
        }
    }
]

for collection in [user_labeled_svm_collection, user_predicted_svm_collection]:
    collection.aggregate(pipeline)

### Merge les données labélisées du Kmeans dans users_compare_both

In [ ]:
pipeline = [
    {
        '$project': {
            '_id': 1,
            'user_id': 1,
            'label_kmeans': 1,
            'CP1': 1,
            'CP2': 1,
            'CP3': 1,
            'CP4': 1,
            'CP5': 1
        }
    },
    {
        '$merge': {
            'into': 'users_compare_both',
            'on': 'user_id',
            'whenMatched': 'merge',
            'whenNotMatched': 'discard'
        }
    }
]

user_predicted_kmeans_collection.aggregate(pipeline)